# Reading Data

In [0]:
df = spark.read.format('parquet')\
    .option('inferSchema', 'true')\
    .load('abfss://bronze@sacarsdatalake.dfs.core.windows.net/rawdata/')

In [0]:
display(df)

# Data transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = df.withColumn("model_category", split(col('Model_ID'),'-')[0])
display(df)

In [0]:
df.withColumn("Units_Sold", col("Units_Sold").cast(StringType())).display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("RevenuePerUnit",col('Revenue')/col('Units_Sold'))
display(df)

In [0]:
display(df.groupBy("Year", "BranchName").agg(sum("Units_Sold").alias("Total_Units")).sort("Year", "Total_Units", ascending=[True,False]))

# Data Writing

In [0]:
df.write.format('parquet')\
    .mode("overwrite")\
    .option('path', 'abfss://silver@sacarsdatalake.dfs.core.windows.net/carsales/')\
    .save()